In [20]:
import pandas as pd
import numpy as np
import PyPDF2 as pdf
import os
import glob
import xlwt
from tqdm import tqdm, trange

# pdfminer.six
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser

In [66]:
# 函數
def extract_text_by_page(pdf_path, maxpages=0): # extract single page
    with open(pdf_path, 'rb') as fh:
        pagecount=0
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True,
                                      maxpages=maxpages):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
 
            text = fake_file_handle.getvalue()
            pagecount+=1
            yield text,pagecount
 
            # close open handles
            converter.close()
            fake_file_handle.close()
def pdf_extractable(pdf_path): # 可否萃取
    with open(pdf_path, 'rb') as fh:
        parser = PDFParser(fh)
        doc = PDFDocument(parser)
        
        # stage 1
        if doc.is_extractable and doc.is_modifiable and doc.is_printable:
            # stage 2
            # 利用 yield 逐頁找尋關鍵字
            for text,page in extract_text_by_page(pdf_path,maxpages=15): 
                if text.find('致股東報告書') != -1:  
                    return True # 都通過才回傳True
    
        return False
    
def getpages_letter_to_stockholder(pdf_path): # 致股東報告書頁碼
    with open(pdf_path, 'rb') as fh:
        parser = PDFParser(fh)
        doc = PDFDocument(parser)
        content = 0
        startpage = 0
        endpage = 0
    
        for text,page in extract_text_by_page(pdf_path, maxpages=20):
            if content == 0:
                if text.find('致股東報告書') != -1 and text.find('公司簡介') != -1 and text.find('公司治理') != -1: # found content
                    content = page
                    continue
            elif startpage == 0:
                if text.find('致股東報告書') != -1:
                    startpage = page
                    continue
            elif endpage == 0:
                if text.find('公司簡介') != -1:
                    endpage=page
                    
            if content != 0 and startpage != 0 and endpage != 0:
                if content<startpage<endpage:
                    return [startpage, endpage]
    return ['None', 'None']

In [22]:
dirPathPattern = r"C:\Users\DC\辰瑜的論文\年報108-109年\*\*\*" 
result = glob.glob(dirPathPattern)

In [23]:
#全部檔案路徑
for i in range(len(result)):
    print(i," ",result[i])

0   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1101\2019_1101_20200609F04_20211212_154148.pdf
1   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1101\2020_1101_20210611F04_20211212_153959.pdf
2   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102\2019_1102_20200623F04_20211212_154547.pdf
3   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102\2020_1102_20210625F04_20211212_154258.pdf
4   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103\2019_1103_20200622F04_20211212_155216.pdf
5   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103\2020_1103_20210617F04_20211212_155303.pdf
6   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1104\2019_1104_20200615F04_20211212_155411.pdf
7   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1104\2020_1104_20210628F04_20211212_155453.pdf
8   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108\2019_1108_20200618F04_20211212_155632.pdf
9   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108\2020_1108_20210616F04_20211212_155819.pdf
10   C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1109\2019_1109_20200623F04_20211212_155859.pdf
11   C:\U

1325   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3060\2020_3060_20210611F04_20211215_222434.pdf
1326   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3071\2019_3071_20200623F04_20211215_222542.pdf
1327   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3071\2020_3071_20210622F04_20211215_222650.pdf
1328   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3088\2019_3088_20200528F04_20211215_222757.pdf
1329   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3088\2020_3088_20210528F04_20211215_222905.pdf
1330   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3211\2019_3211_20200609F04_20211215_223037.pdf
1331   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3211\2020_3211_20210607F04_20211215_223109.pdf
1332   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3213\2019_3213_20200618F04_20211216_104312.pdf
1333   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3213\2020_3213_20210610F04_20211216_104359.pdf
1334   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3217\2019_3217_20200610F04_20211215_223447.pdf
1335   C:\Users\DC\辰瑜的論文\年報108-109年\M2325電腦及週邊\3217\2020_321

2658   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2548\2019_2548_20200526F04_20211217_185146.pdf
2659   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2548\2020_2548_20210525F04_20211217_185217.pdf
2660   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2596\2019_2596_20200618F04_20211217_185248.pdf
2661   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2596\2020_2596_20210616F04_20211217_185320.pdf
2662   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2597\2019_2597_20200527F04_20211217_185351.pdf
2663   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2597\2020_2597_20210528F04_20211217_185425.pdf
2664   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2841\2019_2841_20200630F04_20211217_185459.pdf
2665   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2841\2020_2841_20210630F04_20211217_185530.pdf
2666   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2923\2019_2923_20200630F04_20211217_185602.pdf
2667   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\2923\2020_2923_20210625F04_20211217_185633.pdf
2668   C:\Users\DC\辰瑜的論文\年報108-109年\M2500建材營造\3052\2019_3052_20200619F

In [24]:
#全部檔案名
for i in range(len(result)):
    pdfname = list(os.path.split(result[i]))
    print(pdfname[-1])

2019_1101_20200609F04_20211212_154148.pdf
2020_1101_20210611F04_20211212_153959.pdf
2019_1102_20200623F04_20211212_154547.pdf
2020_1102_20210625F04_20211212_154258.pdf
2019_1103_20200622F04_20211212_155216.pdf
2020_1103_20210617F04_20211212_155303.pdf
2019_1104_20200615F04_20211212_155411.pdf
2020_1104_20210628F04_20211212_155453.pdf
2019_1108_20200618F04_20211212_155632.pdf
2020_1108_20210616F04_20211212_155819.pdf
2019_1109_20200623F04_20211212_155859.pdf
2020_1109_20210624F04_20211212_155958.pdf
2019_1110_20200620F04_20211212_160051.pdf
2020_1110_20210625F04_20211212_160217.pdf
2019_1201_20200623F04_20211212_160341.pdf
2020_1201_20210623F04_20211212_160526.pdf
2019_1203_20200623F04_20211212_160843.pdf
2020_1203_20210623F04_20211212_161121.pdf
2019_1210_20200605F04_20211212_161313.pdf
2020_1210_20210611F04_20211212_161713.pdf
2019_1213_20200616F04_20211212_161938.pdf
2020_1213_20210607F04_20211212_162038.pdf
2019_1215_20200623F04_20211212_163225.pdf
2020_1215_20210624F04_20211212_163

In [55]:
file = result[5]
file_split = list(os.path.split(file))
file_split

['C:\\Users\\DC\\辰瑜的論文\\年報108-109年\\M1100水泥工業\\1103',
 '2020_1103_20210617F04_20211212_155303.pdf']

In [30]:
file_split[-1]

'2019_1101_20200609F04_20211212_154148.pdf'

In [76]:
workbook = xlwt.Workbook(encoding='utf-8')       #新建工作簿
sheet1 = workbook.add_sheet("測試表格")     #新建sheet

title = ['co_id','year','ext','filename','isedible','startpage','endpage','inputpath']
for i in range(0,len(title)):
    sheet1.write(0,i,title[i])
    
for i in tqdm(range(0,len(result))):
    try:
        file = result[i]
        file_split = list(os.path.split(file))
        extract_text_by_page(file)
        isedible = pdf_extractable(file)
        page = getpages_letter_to_stockholder(file)
        
        data = [str(file_split[-1][5:9]),str(int(file_split[-1][0:4])-1911),file_split[-1].split('.')[1],file_split[-1],isedible,page[0],page[1],file]
        for j in range(0,len(data)):
            sheet1.write(i+1,j,data[j])    
    except Exception as e:
        pass
    continue

workbook.save(r'C:\Users\DC\辰瑜的論文\test.xlsx')
pd.read_excel('test.xlsx')

100%|████████████████████████████████████████████████████████████████████████████| 3389/3389 [6:39:43<00:00,  7.08s/it]


,co_id,year,ext,filename,isedible,startpage,endpage,inputpath
0,1101.0,108.0,pdf,2019_1101_20200609F04_20211212_154148.pdf,0.0,None,None,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1101\20...
1,1101.0,109.0,pdf,2020_1101_20210611F04_20211212_153959.pdf,0.0,None,None,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1101\20...
2,1102.0,108.0,pdf,2019_1102_20200623F04_20211212_154547.pdf,1.0,5,7,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102\20...
3,1102.0,109.0,pdf,2020_1102_20210625F04_20211212_154258.pdf,1.0,5,11,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102\20...
4,1103.0,108.0,pdf,2019_1103_20200622F04_20211212_155216.pdf,1.0,4,5,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103\20...
...,...,...,...,...,...,...,...,...
3384,9120.0,109.0,pdf,2020_912000_20210603F04_20211220_192748.pdf,0.0,None,None,C:\Users\DC\辰瑜的論文\年報108-109年\W91存託憑證\912000\20...
3385,9136.0,108.0,pdf,2019_9136_20200527F04_20211220_191847.pdf,0.0,None,None,C:\Users\DC\辰瑜的論文\年報108-109年\W91存託憑證\9136\2019...
3386,9136.0,109.0,pdf,2020_9136_20210514F04_20211220_191928.pdf,0.0,None,None,C:\Users\DC\辰瑜的論文\年報108-109年\W91存託憑證\9136\2020...
3387,9188.0,108.0,pdf,2019_9188_20200618F04_20211220_192027.pdf,0.0,None,None,C:\Users\DC\辰瑜的論文\年報108-109年\W91存託憑證\9188\2019...


In [92]:
data

['9188',
 '109',
 'pdf',
 '2020_9188_20210623F04_20211220_192059.pdf',
 False,
 'None',
 'None',
 'C:\\Users\\DC\\辰瑜的論文\\年報108-109年\\W91存託憑證\\9188\\2020_9188_20210623F04_20211220_192059.pdf']

In [81]:
pd.read_excel('test.xlsx')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3389 entries, 0 to 3388
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   co_id      2797 non-null   float64
 1   year       2797 non-null   float64
 2   ext        2797 non-null   object 
 3   filename   2797 non-null   object 
 4   isedible   2797 non-null   float64
 5   startpage  2797 non-null   object 
 6   endpage    2797 non-null   object 
 7   inputpath  2797 non-null   object 
dtypes: float64(3), object(5)
memory usage: 211.9+ KB


In [113]:
df = pd.read_excel('test.xlsx')
df['isedible'] = df['isedible'].map({0:False,1:True})
df.drop(['inputpath'],axis=1,inplace = True)
df['co_id'] = df['co_id'].astype(str)
df['year'] = df['year']+1911

In [114]:
df['co_id'] = df['co_id'].astype(str)

In [115]:
df

,co_id,year,ext,filename,isedible,startpage,endpage
0,1101.0,2019.0,pdf,2019_1101_20200609F04_20211212_154148.pdf,False,None,None
1,1101.0,2020.0,pdf,2020_1101_20210611F04_20211212_153959.pdf,False,None,None
2,1102.0,2019.0,pdf,2019_1102_20200623F04_20211212_154547.pdf,True,5,7
3,1102.0,2020.0,pdf,2020_1102_20210625F04_20211212_154258.pdf,True,5,11
4,1103.0,2019.0,pdf,2019_1103_20200622F04_20211212_155216.pdf,True,4,5
...,...,...,...,...,...,...,...
3384,9120.0,2020.0,pdf,2020_912000_20210603F04_20211220_192748.pdf,False,None,None
3385,9136.0,2019.0,pdf,2019_9136_20200527F04_20211220_191847.pdf,False,None,None
3386,9136.0,2020.0,pdf,2020_9136_20210514F04_20211220_191928.pdf,False,None,None
3387,9188.0,2019.0,pdf,2019_9188_20200618F04_20211220_192027.pdf,False,None,None


In [116]:
df_pagedata = df[(df['startpage'] != 'None')&(df['endpage'] != 'None')&(df['isedible']==True)]

In [123]:
df_pagedata['co_id'].astype(str)

2       1102.0
3       1102.0
4       1103.0
5       1103.0
8       1108.0
         ...  
3361    9942.0
3362    9944.0
3364    9945.0
3365    9945.0
3367    9955.0
Name: co_id, Length: 1712, dtype: object

In [118]:
df_pagedata.to_excel(r'C:\Users\DC\辰瑜的論文\頁碼簿.xlsx',index = None)